[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aynetdia/Budget_Constrained_Bidding/blob/master/Tutorial.ipynb) 

# Setting up the notebook in Colab

In [ ]:
# Mount on GDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cloning the repo (only when setting up for the first time)
%cd drive/MyDrive
!git clone https://github.com/aynetdia/budget_constrained_bidding.git

Here's a link to the dataset: https://drive.google.com/drive/folders/1YYyxGMDW0EuZA2BI-uR_2j60lpngKgy8?usp=sharing

In order to set up the dataset, you either have to download the linked folder containing the dataset and put it into `/budget_constrained_biddig/data/ipinyou/`, in case you choose to run the notebook locally, or add a GDrive shortcut (go to: Shared with me -> Right click on the dataset folder -> Add shortcut to Drive) and select the  same `/budget_constrained_biddig/data/ipinyou/` as a location to place the shortcut in, if you want to run the notebook in Colab.

In [ ]:
# Go into the project folder and pull the last changes (do that every time before running the notebook)
# If executing after cloning the repo: %cd budget_constrained_bidding
%cd /content/drive/MyDrive/budget_constrained_bidding
!git pull

# Table of Contents

&emsp;&emsp;1. Introduction into the application domain

&emsp;&emsp;2. Methods

&emsp;&emsp;3. DRLB Framework

&emsp;&emsp;4. Description of the Data Set

&emsp;&emsp;5. Implementation

&emsp;&emsp;6. Results 

&emsp;&emsp;7. References

# 1. Introduction

&emsp;Allocating budget efficiently is a problem that has received a lot of attention in marketing literature.
Budget constrained bidding, as an automated bidding strategy and a common scenario in Real-Time Bidding (RTB), is an important concept in online display advertising. In the context of online marketing strategies, budget constrained bidding serves as an important mechanism in which a typical optimization goal for advertisers is to maximize the total value of winning impressions under a certain budget constraint. A winning impression value is usually associated with the expected outcome of an advertisement, e.g. a click or conversion as user feedback. In RTB advertisers give a bid for every individual impression of their advertisements instead of settling for a predefined fixed bid for each campaign. RTB enables advertisers to spend campaigns budget on high-quality ad impressions in order to obtain that positive user feedback. Thus, one challenge is to capture those valuable ad impressions in real-time, which can be solved by an appropriate bidding strategy.
    




### Budget Constrained Bidding

&emsp;A bidding strategy is based on the evaluation of impressions on an ad campaign, e.g. by the predicted click-through rate (CTR) or calculated bid price for each impression. A campaign runs during a certain time period that captures $N$ impression opportunities in sequential order $i$. Considering the competition with other bidders in real-time and the impression value $v{_i}$, an advertiser gives a bid $b{_i}$ and only wins the auction if $b{_i}$ is the highest bid [Wu D., et al., 2018]. In this case the advertiser gets the opportunity to display the ad and to gain winning impressions which then induces costs $c{_i}$.
However, budget constrained bidding is a knapsack problem [Zhou Y., et al., 2008]. An optimal bid in RTB with budget constraint B takes the from of $ b{_i} = v / λ $ , where $v$ denotes the impression value and $λ$ a scaling parameter mapping v to a scale of bid. If $λ$ increases, then the bid price decreases. Hence, high constrained budget should cause the general bidding price level to be lower [Zhang W., et al., 2014].
Bidding strategies usually consider the bid decision by treating each ad impression independently, ignoring the impacts of the overall effectiveness of the campaign and the dynamics of the RTB environment. Therefore, developing an optimal bidding strategy is complex to derive but yet highly important since in RTB only the advertiser with the highest bid wins the auction. Reinforcement Learning (RL) is a solution for the above problem.  


&emsp;This tutorial presents a solution for the budget constrained bidding problem, which leverages model-free reinforcement learning. In this model-free reinforcement learning, state spaces are the impressions feature parameter as well as the auction information and the action is to set a bidding price. The tutorial is based on the findings of Wu D., et al. [2018] presented in „Budget Constrained Bidding by Model-free Reinforcement Learning in Display Advertising“.
In this tutorial, we first introduce basic concepts of Reinforcement Learning, Constrained Markov Decision Process and Deep Q-Networks. In the next section we explain the Deep Reinforcement Learning to Bid framework, followed by a brief description of the dataset and an explanation on the implementation of the algorithms. Lastly, we discuss the results. 
 

# 2. Methods


## 2.1 Introduction to Reinforcement Learning and methods comparison

### Reinforcement Learning  
&emsp;Reinforcement Learning (RL) is a tool to support sequential decision-making events and is a highly active research field of machine learning. It finds enormous application in e.g. real-time-bidding in advertisement auction.

&emsp;RL is a core control problem in which an agent sequentially interacts with an unknown environment to maximize its cumulative reward [Richard S. and Andrew G., 2018]. It is a simulation of the human learning process in dynamic environments without any supervision. To find the best strategy of reward maximization, the agent continuously interacts with the environment and finds the optimal action under different states. Roughly speaking, the agent’s goal is to get as much reward as possible over the long run.

&emsp;For instance, board game playing is a classical dynamic decision-making process. Players need to interact with their counterparts. It is a procedure filled with immediate rewards, intuitive judgments, and each action will generate an impact in the end. The cumulative reward will decide its win-or-failure [Richard S. and Andrew G., 2018].

&emsp;In reality, such scenarios will consist of several problems that can be solved by a class of solution methods, but not all of them can achieve the rewards maximization objective. In the learning process of RL, each action the agent made, will generate some impact not only on the immediate reward but also on the future states. In other words, RL faces a dynamic decision-making problem, the agent will obtain continuous rewards and punishments in this learning process and modify its behaviors according to the environment's feedbacks and finally to maximize the cumulative reward.

&emsp;In order to make the process more clear, we explain some basic concept of RL as follows:

&emsp;&emsp;Policy: the whole actions the learning agent has taken in a concrete period 

&emsp;&emsp;Agent: the one who takes the action

&emsp;&emsp;Environment: the place where the agent takes action and interacts with him

&emsp;&emsp;Action: the move the agent makes to interact with the environment

&emsp;&emsp;State: a situation in which the agent perceives 

&emsp;&emsp;Reward: feedback of the agent’s action 	

&emsp;The image illustrates the whole process of RL.

![Image 1: Reinforcement Learning Process](https://drive.google.com/uc?export=view&id=1y_z3Xg_QCsK9kkfV3tWhtqJNDmW4PrAJ)

	


### Methods comparison	
&emsp;Besides RL, supervised learning and unsupervised learning are the most classic machine learning methods. In this section, we explain why RL is the best choice for solving dynamic decision-making problems. Thus, the core differences among these three methods are shown in table below.


![Table 1: Machine Learning Methods Comparison](https://drive.google.com/uc?export=view&id=1Af_FgYtCeLyFwuKVm5UV9JrZpQhK1D2w)


&emsp;Supervised learning is mainly used to solve regression and classification problems. It explores the relation among labeled, the target variable, and other input variables and produces a model to predict further category identification.

&emsp;In comparison, unsupervised learning obtains unlabeled observations and searches the hidden structure behind the input data. Both of these methods are not suitable for the desired behavior optimization problem in a dynamic process. Unlike supervised and unsupervised learning, which will not consider the capacity or some constrained environments, reinforcement learning will start with a clear initial setting, goal, and interactive agent. All the sequential action the agent takes will generate some influence in this environment. After a dynamic model training process, the agent obtains more and more rewards as desired by taking optimal actions in order to maximize the cumulative reward. 

&emsp;Reinforcement Learning models problems into a Markov Decision Process and matching the output to the problem. In theory, when we can map our problems into an MDP, we can expect that reinforcement learning can be a useful tool to solve such problems. In the next section, we will explain how it works with Markov Decision Process and give more details about Deep Q-Learning, which is a core concept in our project.




# 2.2 (Constrained) Markov Decision Process  
  
### Markov Decision Process 
&emsp;RL is usually modeled as a Markov Decision Process (MDP). MDP determines how good the agents particular state $s$ is. This is defined by a policy $\pi: S —> P(A)$, i.e. what actions $A$ the agent performs in a particular state $s$ [Singh A., 2019]. The policy is a function that defines the probability distribution over actions $(a∈A)$ for each state $(s∈S)$ which fully determines the behavior of an agent. The agent uses the policy to interact with the environment by sequentially taking actions and to alter its behavior in order to maximize the cumulative reward $R$ (all the rewards that the agent receives from the environment) [Wu D., et al., 2018]. In MDP the state transition $T(s, a, s’)$ produces the probability $Pr(s’ | s, a)$ to transition into state $s’$ given that the agent was in state $s$ and took action $a$. The immediate reward function, which is the reward $r$ an agent receives from the current state, shows the agents immediate reward from the environment. It will make the agent converge to suboptimal solutions, since the agent greeds to take actions to decrease $λ$ in order to gain more immediate reward each time [Wu D., et al., 2018]. If an agent follows a policy $\pi$  at time $t$  then $\pi(a | s)$ is the probability of that agent taking action $a$ at a particular time step $t$ [Singh A., 2019].
The goal of the agent is to learn the optimal policy $\pi$  which maximizes the long-term expected reward form the start state [Wu D., et al., 2018]: $\pi^* = arg max_{\pi}  E [R | \pi]$ , where $R = \sum^T_{t=1} \gamma^{t-1} r_t $ denotes the cumulative discounted reward.





### Bellman Equation for value function

&emsp;The Bellman Equation helps to determine optimal policies and optimal value functions, where value functions differ according to the different policies.
The value function expresses the relationship between the value of a state $s$ and the values of its successor states $s’$ . The value of a state under an optimal policy must equal the expected reward for the best action from that state. It is decomposed into the Immediate Reward, $R[t+1]$ , and the discounted value of successor states [Singh A., 2019]:
$v(s)=E[R_{t+1}+\gamma+v(S_{t+1}) | S{_t} = s]$ .

  


### Constrained Markov Decision Process

&emsp;Constrained Markov Decision Process (CMDP) extends the concept of MDP by adding additional cost constraints, i.e. the total budget [Wu D., et al., 2018 ; Altman E., 1999]: In CMDP the action of the agent is to submit bids to sequential impression opportunities. In an episodic CMDP the agent regulates $λ$ with a fixed time step T until the episode ends. The agent observes states and takes action to adjust $λ_{t-1}$ to $λ_{t}$ during each time step $t$. However, the agent will still converge to the suboptimal solution due to the immediate reward function.
Considering the dynamic and unpredictable RTB environment, makes it hard to obtain the transition dynamic. This is due to the fact that CMDP is usually a model-based RL approach, where the state transition dynamics $T$ needs to be known in advance. Therefore, our focal paper proposes to use deep Q-Networks (DQN) which is a model-free RL approach.

## 2.3 Deep Q-Networks


&emsp;DQN is employed to model budget constrained bidding. It is a popular method of Q-learning in RL [Kim S. et al., 2019, Stekolshchik R., 2020]. 

&emsp;DQN uses deep neural networks to approximate the Q-function (action-value function) and samples random minibatches of transition from experience replay buffer as training data to reduce correlations. Furthermore, the DQN consists of a target network. The target network is a technique to stabilize learning by delaying the update of target values. It is a copy of local network which is held as a constant in order to stable the target for learning for a fixed number of time steps. 

&emsp;In DQN the parameter $\epsilon$ of the $\epsilon$-greedy mechanism controls the proportion of exploration and exploitation, where $\epsilon$ is bounded at 0 < $\epsilon$ < 1. The exploitation describes that the agent finds his following action by maximizing the Q-value over all possible actions for the given state. 
DQN iteratively learns Q-function $Q(s_{t}, a_{t})$ corresponding to the optimal policy. It quantifies the quality of taking action $a_{t}$ at state $s_{t}$ [Wu D., et al., 2018]. The agent uses deep neural networks to determine its actions and to train the neural network in such a way that it predicts the weighted, cumulative rewards of all actions. In DQN the agent learns the optimal policy by interacting directly with the workflow environment and also with the policies of other agents.

# 3. DRLB framework

# 4. Description of the Data Set

# 5. Implementation

In [ ]:
#you should find yourself inside the project folder after you've pulled the latest changes
!ls

In [ ]:
# import the dataset
import pandas as pd
import numpy as np
import os
#if you runnning this notebook locally, uncomment next line
#bid_requests=pd.read_csv(os.getcwd() + r'\data\ipinyou\1458\train.log.txt', sep="\t")
bid_requests = pd.read_csv('data/ipinyou/1458/train.log.txt', sep="\t")

In [15]:
# add the necessary data intervals

def get_time_interval(data):
    time_inv=int(data[10:12])
    if time_inv>=0 and time_inv<15:
            return ("00")
    elif time_inv >= 15 and time_inv < 30:
            return ("15")
    elif time_inv >= 30 and time_inv < 45:
            return ("30")
    elif time_inv >= 45 and time_inv <=60:
            return ("45")
    else:
            return(None)



In [16]:
if 'min' in bid_requests.columns:
    print("min already exists")
    pass
else:
    bid_requests["timestamp"]=bid_requests["timestamp"].apply(str)
    min_intervals = bid_requests.apply(lambda row : get_time_interval(row['timestamp']), axis = 1)
    bid_requests.insert(3, "min", min_intervals) # insert the new column after the 'hour' column
    # save the updated dataset. do not run again!
    bid_requests.to_csv('data/ipinyou/1458/train.log.txt', sep="\t", header=True, index=False)

min already exists


In [17]:
#if error "No module named 'lru'" pop up, uncomment next line
#!pip install  lru-dict

# Train DRLB

import sys  
import torch
import cloudpickle
sys.path.insert(0, 'src/rtb_agent')
from rl_bid_agent import RlBidAgent
import gym, gym_auction_emulator

env = gym.make('AuctionEmulator-v0')
env.seed(0)
agent = RlBidAgent()

obs, done = env.reset()
train_budget = env.bid_requests.payprice.sum()/8

budget_proportions = []
for episode in env.bid_requests.weekday.unique():
    budget_proportions.append(len(env.bid_requests[env.bid_requests.weekday == episode])/env.total_bids)
for i in range(len(budget_proportions)):
    budget_proportions[i] = round(train_budget * budget_proportions[i])
agent.episode_budgets = budget_proportions
agent._reset_episode()
agent.cur_day = obs['weekday']
agent.cur_hour = obs['hour']
agent.cur_state = agent._get_state() # observe state s_0

epochs = 400

for epoch in range(epochs):
    while not done:
        bid = agent.act(obs) # obs = state
        next_obs, cur_reward, cur_cost, win, done = env.step(bid)
        agent._update_reward_cost(bid, cur_reward, cur_cost, win)
        obs = next_obs
    print("Total Impressions won {} value = {}".format(agent.total_wins, agent.total_rewards))
    if ((epoch + 1) % 20) == 0:
        PATH = 'models/model_state_{}.tar'.format(epoch)
        torch.save({'local_q_model': agent.dqn_agent.qnetwork_local.state_dict(),
                    'target_q_model':agent.dqn_agent.qnetwork_target.state_dict(),
                    'q_optimizer':agent.dqn_agent.optimizer.state_dict(),
                    'rnet': agent.reward_net.reward_net.state_dict(),
                    'rnet_optimizer': agent.reward_net.optimizer.state_dict()}, PATH)

        f = open('models/rnet_memory_{}.txt'.format(epoch), "wb")
        cloudpickle.dump(agent.dqn_agent.memory, f)
        f.close()
        f = open('models/rdqn_memory_{}.txt'.format(epoch), "wb")
        cloudpickle.dump(agent.reward_net.memory, f)
        f.close()

        pd.DataFrame(agent.step_memory).to_csv('models/step_history_{}.csv'.format(epoch),header=None,index=False)
        agent.step_memory=[]
        pd.DataFrame(agent.episode_memory).to_csv('models/episode_history_{}.csv'.format(epoch),header=None,index=False)
        agent.step_memory=[]

env.close()

ModuleNotFoundError: No module named 'lru'

In [51]:
#random bidding
class Random_Bidding():


        def random_bidding(self,highest_bid):

                #generate a random number generator (0,300)
                #get a dataframe with columns('click', 'slotprice', 'payprice','r_bid_price','wins')
                c_names=('click','day', 'slotprice', 'payprice','r_bid_price','wins')
                zero_Data=np.zeros(shape=(self.total_bids,len(c_names)))
                self.df=pd.DataFrame(zero_Data,columns=c_names)
                self.df['day']=self.bid_requests['weekday']
                self.df['click']=self.bid_requests['click']
                self.df['slotprice'] = self.bid_requests['slotprice']
                self.df['payprice'] = self.bid_requests['payprice']
                self.df['r_bid_price']=np.random.randint(0,highest_bid,[self.total_bids,1])


                self.rem_budget=self.budget
                self.cur_day=str(int(self.df['day'][0]))
                
                for i in range(len(self.df)-1):
                        if str(int(self.df.loc[i]['day'])) != self.cur_day:
                                self.cur_day=str(int(self.df.loc[i]['day']))
                                self.rem_budget=self.budget

                        cost=self.df.loc[i]['r_bid_price']
                        if self.rem_budget<cost:
                                self.df.loc[i]['r_bid_price']=self.rem_budget
                                cost = self.df.loc[i]['r_bid_price']

                        if self.rem_budget<=0:
                                  self.df.loc[i]['r_bid_price']=0
                                  cost=0

                        self.rem_budget-=(cost/1e9)



                def wins_value(row):
                        if row['r_bid_price'] >= row['slotprice'] and row['r_bid_price'] > row['payprice']:
                                return 1
                        return 0

                self.df['wins'] = self.df.apply(wins_value, axis=1)

In [62]:
#load the test dataset
# if you are running this notebook locally, please uncomment next line
#bid_requests_test=pd.read_csv(os.getcwd() + r'\data\ipinyou\1458\test.log.txt', sep="\t")
bid_requests_test = pd.read_csv('data/ipinyou/1458/test.log.txt', sep="\t")

the two code blocks below base on ['1/2','1/4','1/8','1/16','1/32'] * total cost

In [102]:
bid_random = bid_requests_test
bid_tol_cost= bid_requests_test.payprice.sum()
bid_budget = pd.DataFrame()
bid_budget['bugdet']=[bid_tol_cost/2,bid_tol_cost/4,bid_tol_cost/8,bid_tol_cost/16,bid_tol_cost/32]
bid_budget['portion']=['1/2','1/4','1/8','1/16','1/32']
bid_budget


,bugdet,portion
0,2.260823e+07,1/2
1,1.130411e+07,1/4
2,5.652057e+06,1/8
3,2.826028e+06,1/16
4,1.413014e+06,1/32


In [103]:

rb_result_df = pd.DataFrame(columns= ['day','portion','click','winning_impr'])
highest_bid = bid_random.bidprice.sum()/len(bid_random)
ass_num=0
for i in range(len(bid_budget)):
    budget=bid_budget.at[i,'bugdet']
   # for epoch in range(epochs):
    rb = Random_Bidding()
    rb.budget = budget
    rb.bid_requests=bid_random
    rb.total_bids=len(rb.bid_requests)
    rb.random_bidding(highest_bid=highest_bid)
    ass_num=0
    for day in bid_random['weekday'].unique():
        rb_result_df.at[i*len(bid_random['weekday'].unique())+ass_num,'portion']=bid_budget.at[i,'portion']
        rb_result_df.at[i*len(bid_random['weekday'].unique())+ass_num,'day']=day
        rb_result_df.at[i*len(bid_random['weekday'].unique())+ass_num,'winning_impr']=sum(rb.df.loc[rb.df['day']==day]['wins'])
        rb_result_df.at[i*len(bid_random['weekday'].unique())+ass_num,'click']=sum(rb.df.loc[(rb.df['day']==day) & (rb.df['wins']==1)]['click'])
        ass_num+=1



rb_result_df
    



,day,portion,click,winning_impr
0,4,1/2,120,146726
1,5,1/2,101,153173
2,6,1/2,112,162084
3,4,1/4,128,146863
4,5,1/4,108,153014
5,6,1/4,114,161910
6,4,1/8,113,146633
7,5,1/8,108,152761
8,6,1/8,117,161851
9,4,1/16,114,146757


the two code blocks below base on ['1/2','1/4','1/8','1/16','1/32'] * total cost /3

In [104]:
bid_random = bid_requests_test
bid_tol_cost= bid_requests_test.payprice.sum()
bid_budget = pd.DataFrame()
bid_budget['bugdet']=[bid_tol_cost/2,bid_tol_cost/4,bid_tol_cost/8,bid_tol_cost/16,bid_tol_cost/32]
bid_budget['bugdet']/=3
bid_budget['portion']=['1/2','1/4','1/8','1/16','1/32']
bid_budget


,bugdet,portion
0,7.536076e+06,1/2
1,3.768038e+06,1/4
2,1.884019e+06,1/8
3,9.420095e+05,1/16
4,4.710047e+05,1/32


In [105]:

rb_result_df_b = pd.DataFrame(columns= ['day','portion','click','winning_impr'])
highest_bid = bid_random.bidprice.sum()/len(bid_random)
ass_num=0
for i in range(len(bid_budget)):
    budget=bid_budget.at[i,'bugdet']
   # for epoch in range(epochs):
    rb = Random_Bidding()
    rb.budget = budget
    rb.bid_requests=bid_random
    rb.total_bids=len(rb.bid_requests)
    rb.random_bidding(highest_bid=highest_bid)
    ass_num=0
    for day in bid_random['weekday'].unique():
        rb_result_df_b.at[i*len(bid_random['weekday'].unique())+ass_num,'portion']=bid_budget.at[i,'portion']
        rb_result_df_b.at[i*len(bid_random['weekday'].unique())+ass_num,'day']=day
        rb_result_df_b.at[i*len(bid_random['weekday'].unique())+ass_num,'winning_impr']=sum(rb.df.loc[rb.df['day']==day]['wins'])
        rb_result_df_b.at[i*len(bid_random['weekday'].unique())+ass_num,'click']=sum(rb.df.loc[(rb.df['day']==day) & (rb.df['wins']==1)]['click'])
        ass_num+=1



rb_result_df_b
    

,day,portion,click,winning_impr
0,4,1/2,120,146896
1,5,1/2,115,153092
2,6,1/2,109,162087
3,4,1/4,114,146919
4,5,1/4,101,152920
5,6,1/4,111,162273
6,4,1/8,104,146596
7,5,1/8,103,152858
8,6,1/8,108,162084
9,4,1/16,129,146888


# 6. Results

# 7. References 

Altman E. (1999): „Constrained Markov decision processes“, CRC Press, Vol. 7.  

Kim S., Asadi K., Littman M., et al. (2019): „Removing the Target Network from Deep Q-Networks with the Mellowmax Operator“ , Proc. of the 18th International Conference on Autonomous Agents and Multiagent Systems.

Singh A. (2019): „Reinforcement Learning : Markov-Decision Process (Part 1)“, [online] https://towardsdatascience.com/introduction-to-reinforcement-learning-markov-decision-process-44c533ebf8da [08.12.2020].  

Stekolshchik R. (2020): „A pair of interrelated neural networks in Deep Q-Network“, [online] https://towardsdatascience.com/a-pair-of-interrelated-neural-networks-in-dqn-f0f58e09b3c4 [09.12.2020].

Richard S. and Andrew G. (2018): "Reinforcement Learning", The MIT Press,pp.1-6. 

Wu D., Chen X., Yang X., et al. (2018): „Budget Constrained Bidding by model-free Reinforcement Learning in Display Advertising“, Association for Computing Machinery, 27, pp. 1443–1451.  

Zhang W., Yuan S., and Wang J. (2014): „Optimal Real-Time Bidding for Display Advertising“ Association for Computing Machinery, 20, pp. 1077–1086.  

Zhou Y., Chakrabarty D., and Lukose R. (2008): „Budget constrained bidding in keyword auctions and online knapsack problems“, International Workshop on Internet and Network Economics, Springer, pp. 566–576.  

# 7. References 

Altman E. (1999): „Constrained Markov decision processes“, CRC Press, Vol. 7.  

Kim S., Asadi K., Littman M., et al. (2019): „Removing the Target Network from Deep Q-Networks with the Mellowmax Operator“ , Proc. of the 18th International Conference on Autonomous Agents and Multiagent Systems.

Singh A. (2019): „Reinforcement Learning : Markov-Decision Process (Part 1)“, [online] https://towardsdatascience.com/introduction-to-reinforcement-learning-markov-decision-process-44c533ebf8da [08.12.2020].  

Stekolshchik R. (2020): „A pair of interrelated neural networks in Deep Q-Network“, [online] https://towardsdatascience.com/a-pair-of-interrelated-neural-networks-in-dqn-f0f58e09b3c4 [09.12.2020].

Richard S. and Andrew G. (2018): "Reinforcement Learning", The MIT Press,pp.1-6. 

Wu D., Chen X., Yang X., et al. (2018): „Budget Constrained Bidding by model-free Reinforcement Learning in Display Advertising“, Association for Computing Machinery, 27, pp. 1443–1451.  

Zhang W., Yuan S., and Wang J. (2014): „Optimal Real-Time Bidding for Display Advertising“ Association for Computing Machinery, 20, pp. 1077–1086.  

Zhou Y., Chakrabarty D., and Lukose R. (2008): „Budget constrained bidding in keyword auctions and online knapsack problems“, International Workshop on Internet and Network Economics, Springer, pp. 566–576.  